In [1]:
from cobra.io import read_sbml_model, write_sbml_model
from mewpy.cobra.util import add_enzyme_constraints
from mewpy.simulation import get_simulator
from mewpy.simulation.environment import Environment as Environment
from mewpy.util.request import retreive_gene,retreive_protein
from tqdm.auto import tqdm
import json

In [2]:
model = read_sbml_model("../models/non-ec/ncbi_refseq/vpi5482.xml")
sim = get_simulator(model)
sim.set_objective("Growth")

Set parameter Username


In [3]:
sim.simulate()

objective: 57.923433809721274
Status: OPTIMAL
Method:SimulationMethod.FBA

### Retreive MW infos

The protein IDs and MW (Da) are obtained from UniProt.

Note: does not retreive Kcats from BRENDA. All catalytic rates are equal to 100000/s 

In [4]:
data_all = dict()
try:
    with open(sim.id+".json", 'r') as f:
        data = json.load(f)
except:
    data = dict()
    for gene in tqdm(sim.genes):
        try:
            info = retreive_gene(gene,"Escherichia coli")
            # set the inverse kcat to a low value
            data[gene]={'protein':info['protein'],'mw':info['mw']*0.001,'kcat':1e-5}
            data_all[gene]=info
        except:
            data[gene]={'protein': gene[len(sim._g_prefix):], 'mw': 0, 'kcat': 0}
    with open(sim.id+".json", "w") as f:
        json.dump(data,f)

  0%|          | 0/634 [00:00<?, ?it/s]

In [5]:
print("Genes with missing data:")
for k,v in data.items():
    if v['mw']==0:
        print(k)

Genes with missing data:
WP_008763097_1
WP_011108119_1
WP_008761948_1
WP_008765576_1
WP_008766783_1
WP_011107667_1
WP_048696927_1
WP_011108281_1
WP_011108350_1
WP_008759987_1
WP_008762840_1
WP_008766155_1
WP_011109383_1
WP_008761424_1
spontaneous
WP_011107789_1
WP_008761157_1
WP_008762326_1
WP_008767599_1
WP_008761034_1
WP_008762632_1
WP_008766152_1
WP_008760452_1
WP_008763249_1
WP_008763251_1
WP_008763252_1
WP_011108124_1
WP_008762695_1
WP_011108532_1
WP_008762692_1
WP_008762708_1
WP_011107844_1
WP_008761370_1
WP_008759603_1
WP_008765632_1
WP_008766068_1
WP_008763094_1
WP_008764627_1
WP_008760056_1
WP_048695834_1
WP_008760003_1
WP_008762874_1
WP_011107258_1
WP_011107257_1
WP_008760694_1
WP_008759608_1
WP_008760796_1
WP_008763215_1
WP_008767816_1
WP_008765558_1
WP_011108898_1
WP_032840465_1
WP_008762567_1
WP_008765713_1
WP_008760542_1
WP_008761922_1
WP_008762696_1
WP_055229256_1
WP_011107372_1
WP_008761151_1
WP_008763773_1
WP_008762746_1
WP_008765038_1
WP_008763604_1
WP_011108698_1
WP_

In [6]:
print(data)

{'WP_008763097_1': {'protein': 'WP_008763097_1', 'mw': 0, 'kcat': 0}, 'WP_011108119_1': {'protein': 'WP_011108119_1', 'mw': 0, 'kcat': 0}, 'WP_008761948_1': {'protein': 'WP_008761948_1', 'mw': 0, 'kcat': 0}, 'WP_008765576_1': {'protein': 'WP_008765576_1', 'mw': 0, 'kcat': 0}, 'WP_008766783_1': {'protein': 'WP_008766783_1', 'mw': 0, 'kcat': 0}, 'WP_011107667_1': {'protein': 'WP_011107667_1', 'mw': 0, 'kcat': 0}, 'WP_048696927_1': {'protein': 'WP_048696927_1', 'mw': 0, 'kcat': 0}, 'WP_011108281_1': {'protein': 'WP_011108281_1', 'mw': 0, 'kcat': 0}, 'WP_011108350_1': {'protein': 'WP_011108350_1', 'mw': 0, 'kcat': 0}, 'WP_008759987_1': {'protein': 'WP_008759987_1', 'mw': 0, 'kcat': 0}, 'WP_008762840_1': {'protein': 'WP_008762840_1', 'mw': 0, 'kcat': 0}, 'WP_008766155_1': {'protein': 'WP_008766155_1', 'mw': 0, 'kcat': 0}, 'WP_011109383_1': {'protein': 'WP_011109383_1', 'mw': 0, 'kcat': 0}, 'WP_008761424_1': {'protein': 'WP_008761424_1', 'mw': 0, 'kcat': 0}, 'spontaneous': {'protein': 'spont

### Build the ec model

In [ ]:
ec_sim = add_enzyme_constraints(sim, data)

### Protein pool usage

In [ ]:
res = ec_sim.simulate()
print(res)

In [ ]:
res.find('pool')

### Save the model

In [ ]:
write_sbml_model(ec_sim.model,"eciAF1260.xml")

In [ ]:
env = Environment.complete(ec_sim, max_uptake=1000.0, inplace=False)

In [ ]:
env['prot_pool_exchange']=(0,0.8)

In [ ]:
ec_sim.simulate(constraints=env)